# Introduction

## Imports

In [31]:
import numpy as np
import matplotlib.pyplot as plt
import gymnasium as gym
import tqdm

## Env

In [4]:
env = gym.make('Taxi-v3')

## Constants

In [27]:
num_states = env.observation_space.n
num_actions = env.action_space.n  # number of actions plus options

GAMMA = 0.99
ALPHA = 0.2
EPSILON = 0.1

actions = ["south", "north", "east", "west", "pickup", "dropoff"]
color = ["red", "green", "blue", "yellow", "taxi"]
positions = [(0, 4), (4, 4), (3, 0), (0, 0)]

## Options

In [28]:
def epsilon_greedy(Q, state):
    if np.random.uniform(0, 1) < EPSILON:
        return np.random.choice(Q.shape[1])
    else:
        return np.argmax(Q[state])

In [29]:
# Does epsilon greedy on the state actions values until it reaches a colored state.

def Red(env, state, Q):
    optact = None
    optdone = False
    x, y, pos, dest = env.unwrapped.decode(state)
    if (x, y) == positions[0]:
        optdone = True
        optact = 4
    else:
        optact = epsilon_greedy(Q, state)
    return optact,optdone
def Green(env, state, Q):
    optact = None
    optdone = False
    x, y, pos, dest = env.unwrapped.decode(state)
    if (x, y) == positions[1]:
        optdone = True
        optact = 4
    else:
        optact = epsilon_greedy(Q, state)
    return optact,optdone
def Blue(env, state, Q):
    optact = None
    optdone = False
    x, y, pos, dest = env.unwrapped.decode(state)
    if (x, y) == positions[2]:
        optdone = True
        optact = 4
    else:
        optact = epsilon_greedy(Q, state)
    return optact,optdone
def Yellow(env, state, Q):
    optact = None
    optdone = False
    x, y, pos, dest = env.unwrapped.decode(state)
    if (x, y) == positions[3]:
        optdone = True
        optact = 4
    else:
        optact = epsilon_greedy(Q, state)
    return optact,optdone

        

## SMDP